In [1]:
# We import the libraries that we will need.
import numpy as np
from collections import Counter
from random import choices
from tqdm import tqdm

In [2]:
def create_genome(gen_size):
    genome = [] # Empty list to store the genome.
    genome.append(int(np.random.randint(cityn+1))) # Random integer to the genome list.
    # Iterate over the range of gen_size.
    for i in range(gen_size):
        flag = True
        dobreak = False
        while flag:
            j  = int(np.random.randint(0, cityn))
            # Check if there is an adjacency between the last element in the genome and the randomly generated integer.
            if adj[genome[-1], j] == 1:
                flag = False
            # Check if the number of cities in the genome list is equal to city_no + 1.
            if cities([genome])[0] == cityn+1:
                dobreak = True
        if dobreak: break
        genome.append(j) # Add the generated integer to the genome list.
    return genome

def solution(genome):
    solution = ""
    # Iterate over the range of the length of the genome list, excluding the last element.
    for i in range(len(genome)-1):
        solution += f"{citiesname[genome[i]]} -> "
    # Name of the city corresponding to the last element.
    solution += f"{citiesname[genome[-1]]}"
    return solution  # Return the generated solution

def fitness(genome):
    cost = 0
    time = 0  
    for i in range(len(genome)-1):
        if genome[i] == genome[i+1]:
            break  # Exit the loop if the same city is visited consecutively
        else:
            cost += dist_cost[genome[i]][genome[i+1]]  # Add the distance cost between consecutive cities.
            time += time_cost[genome[i]][genome[i+1]]  # Add the time cost between consecutive cities.
    if time > max_t:
        return 0 
    return cost  # Return the total cost of the genome.

def first_generation(pop_size, gen_size):
    # Create an empty list to store the initial generation
    generation = []
    # Generate random genomes and add them to the generation
    for i in range(pop_size):
        generation.append(create_genome(gen_size))
    # Return the generated generation
    return generation
    
def next_generation(pop, pop_size, elitism_coef):
    fit = pop_fitness(pop)
    # Check if all fitness values are 0 or the sum of fitness values is 0
    if np.all(fit == 0) or sum(fit) == 0:
        return first_generation(pop_size, gen_size)
    citiesno = cities(pop)
    temp = pop
    temp.sort(key=fitness, reverse=True)
    # Adjust fitness values by adding the number of cities visited
    for i in range(len(fit)):
        if fit[i] != 0:
            fit[i] += citiesno[i]
    nextgen = []
    # Select the top individuals (elitism) for the next generation
    for i in range(elitism_coef):
        nextgen.append(temp[i])
    # Perform mating and mutation to generate the remaining individuals
    for i in range(int(np.floor((pop_size - elitism_coef) / 2))):
        pair = choices(pop, weights=fit, k=2)
        nextgen.extend(mutate(pair[0], pair[1]))
    return nextgen

def pop_fitness(population):
    fit = []
    # Calculate fitness value for each individual in the population
    for x in population:
        fit.append(fitness(x))
    totcost = 0
    for i in range(len(fit)):
        totcost += fit[i]
    cont = 0
    # Count the number of individuals with non-zero fitness
    for i in range(len(fit)):
        if fit[i] != 0:
            cont += 1
    # Normalize fitness values based on the total cost
    for i in range(len(fit)):
        if cont == 1:
            fit[i] = fit[i] / (totcost)
        elif (cont > 1):
            fit[i] = (1 - (fit[i] / totcost)) / (cont - 1)
    return fit

def cities(pop):
    cit = []  # List to store the number of unique cities in each genome
    for genome in pop:
        counter_object = Counter(genome)  # Count occurrences of each city in the genome
        keys = counter_object.keys()  # Get unique cities from the counter object
        cit.append(len(keys))  # Append count of unique cities to the list
    return cit  # Return list of counts for unique cities in each genome
        
def mutate(genome1, genome2):
    # Check if there are no common cities between the genomes.
    if set(genome1).intersection(genome2) == set():
        # If there are no common cities, create new children genomes.
        child1 = create_genome(len(genome1))
        child2 = create_genome(len(genome2))
        return child1, child2
    while True:
        city = int(np.random.randint(0, cityn))
        # Check if the chosen city is present in both genomes
        if (city in genome1) and (city in genome2):
            # Perform crossover by swapping subsequences after the chosen city
            p = genome1.index(city)
            q = genome2.index(city)
            child1 = genome1[:p] + genome2[q:]
            child2 = genome2[:q] + genome1[p:]
            return child1, child2

In [3]:
#We define the maximum time as 72 hours, that is, 4320 minutes.
max_t = 72*60

#We establish the name of the cities to then create the matrices.
citiesname = ['Madrid', 'Barcelona', 'Paris', 'Lyon', 'London', 'Brussels', 'Frankfurt', 'Milan', 'Amsterdam', 'Cologne', 'Berlin', 'Roma']

#Adjacency matrix, reflects the connections between cities.
adj = np.asarray([
    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], 
    [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0], 
    [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0], 
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 
])

# Matrix of the time to travel between cities.
time_cost = np.asarray([
    [0, 150, 225, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [150, 0, 390, 200, 0, 0, 0, 0, 0, 0, 0, 0], 
    [225, 390, 0, 112, 136, 82, 480, 0, 0, 0, 0, 0], 
    [0, 200, 112, 0, 0, 0, 0, 176, 0, 0, 0, 0], 
    [0, 0, 136, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 82, 0, 0, 0, 0, 0, 120, 0, 0, 0],
    [0, 0, 480, 0, 0, 0, 0, 454, 0, 120, 232, 0], 
    [0, 0, 0, 176, 0, 0, 454, 0, 0, 0, 0, 168], 
    [0, 0, 0, 0, 0, 120, 0, 0, 0, 120, 364, 0], 
    [0, 0, 0, 0, 0, 0, 120, 0, 120, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 232, 0, 364, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 168, 0, 0, 0, 0], 
])

# Matrix of costs to travel between cities.
dist_cost = np.asarray([
    [0, 98, 380, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [98, 0, 400, 320, 0, 0, 0, 0, 0, 0, 0, 0], 
    [380, 400, 0, 185, 98, 80, 345, 0, 0, 0, 0, 0],
    [0, 320, 185, 0, 0, 0, 0, 180, 0, 0, 0, 0], 
    [0, 0, 98, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 80, 0, 0, 0, 0, 0, 48, 0, 0, 0], 
    [0, 0, 345, 0, 0, 0, 0, 240, 0, 40, 125, 0], 
    [0, 0, 0, 180, 0, 0, 240, 0, 0, 0, 0, 125],
    [0, 0, 0, 0, 0, 48, 0, 0, 0, 40, 235, 0], 
    [0, 0, 0, 0, 0, 0, 40, 0, 40, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 125, 0, 235, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 125, 0, 0, 0, 0], 
])

In [4]:
cityn = len(adj) - 1  # Number of cities: 'adj' list minus 1.
pop_size = 20  
gen_size = 80
epochs = 600  # Number of iterations.
elitism_coef = 2  # Number individuals preserved.

genome = create_genome(gen_size)
sol = solution(genome)  # Convert the genome into a string representation of the solution

print(sol)  # Print the solution string
print(fitness(genome))  # Print the fitness of the genome

Amsterdam -> Cologne -> Frankfurt -> Paris -> Frankfurt -> Paris -> Lyon -> Paris -> Lyon -> Barcelona -> Madrid -> Paris -> Barcelona -> Madrid -> Barcelona -> Lyon -> Barcelona -> Lyon -> Milan -> Frankfurt -> Paris -> London -> Paris -> Lyon -> Milan -> Frankfurt -> Milan -> Lyon -> Barcelona -> Lyon -> Milan -> Lyon -> Milan -> Frankfurt -> Paris -> Madrid -> Barcelona -> Lyon -> Paris -> London -> Paris -> Lyon -> Milan -> Lyon -> Barcelona -> Paris -> London -> Paris -> Barcelona -> Paris -> Lyon -> Paris -> Madrid -> Barcelona -> Paris -> London -> Paris -> Frankfurt -> Cologne -> Amsterdam -> Brussels -> Amsterdam -> Cologne -> Amsterdam -> Berlin -> Amsterdam -> Cologne -> Amsterdam -> Cologne -> Frankfurt -> Paris -> Lyon -> Paris -> Brussels -> Amsterdam -> Brussels -> Paris -> Brussels -> Amsterdam -> Berlin -> Amsterdam
0


In [5]:
genome1 = create_genome(gen_size)  # Random genome for the first parent.
genome2 = create_genome(gen_size)  # Random genome for the second parent.
children = mutate(genome1, genome2)  # Mutated children from the parents.

for child in children:
    print(solution(child))
    print(fitness(child))
    print()

Brussels -> Paris -> Madrid -> Barcelona -> Lyon -> Barcelona -> Madrid -> Paris -> London -> Paris -> Barcelona -> Lyon -> Milan -> Frankfurt -> Berlin -> Amsterdam -> Cologne -> Frankfurt -> Milan -> Frankfurt -> Paris -> Brussels -> Amsterdam -> Berlin -> Amsterdam -> Cologne -> Frankfurt -> Milan -> Lyon -> Milan -> Frankfurt -> Cologne -> Frankfurt -> Paris -> Brussels -> Paris -> Lyon -> Barcelona -> Paris -> Frankfurt -> Berlin -> Frankfurt -> Berlin -> Amsterdam -> Berlin -> Frankfurt -> Berlin -> Amsterdam -> Berlin -> Amsterdam -> Berlin -> Amsterdam -> Berlin -> Frankfurt -> Berlin -> Frankfurt -> Paris -> Frankfurt -> Paris -> Madrid -> Barcelona -> Madrid -> Barcelona -> Lyon -> Barcelona -> Paris -> Madrid -> Paris -> Madrid -> Paris -> Lyon -> Paris -> Barcelona -> Madrid -> Paris -> Frankfurt -> Cologne -> Amsterdam -> Cologne -> Frankfurt -> Paris
0

Barcelona -> Paris -> Madrid -> Barcelona -> Paris -> Madrid -> Barcelona -> Paris -> Brussels -> Paris -> Lyon -> Milan

In [6]:
pop = first_generation(pop_size, gen_size)  # Initial population.
pop = next_generation(pop, pop_size, elitism_coef)  # Next generation of the population.
fit = pop_fitness(pop)  # Fitness values for the population.

# Each genome in the population.
for gene in pop:
    print(gene)

# Fitness value for each genome in the population.
for x in fit:
    print(x)

[7, 3, 1, 0, 2, 3, 2, 5, 8, 10, 8, 10, 6, 10, 6, 10, 6, 2, 6, 10, 8, 10, 8, 10, 6, 7, 3, 7, 3, 1, 3, 7, 6, 7, 3, 1, 0, 1, 0, 2, 5, 2, 5, 8, 5, 8, 9, 8, 10, 6, 7, 6, 2, 1, 2, 3, 2, 6, 2, 6, 10, 6, 9, 6, 7, 6, 7, 3, 7, 6, 7, 6, 7, 3, 7, 6, 9, 6, 2, 4, 2]
[7, 6, 10, 6, 7, 3, 2, 4, 2, 0, 1, 0, 2, 3, 7, 3, 7, 6, 10, 8, 5, 2, 0, 2, 3, 1, 2, 5, 8, 10, 6, 9, 6, 10, 6, 7, 3, 1, 3, 7, 6, 7, 3, 7, 6, 2, 1, 3, 2, 5, 2, 3, 2, 6, 7, 3, 1, 2, 6, 9, 8, 9, 6, 2, 5, 2, 5, 2, 5, 2, 5, 8, 10, 6, 7, 3, 7, 3, 2, 0, 2]
[9, 8, 10, 6, 9, 6, 7, 3, 7, 6, 2, 4, 2, 4, 2, 3, 1, 3, 7, 6, 2, 3, 1, 3, 7, 3, 1, 0, 1, 0, 1, 3, 7, 3, 2, 6, 10, 8, 10, 6, 2, 5, 2, 3, 7, 3, 2, 5, 8, 5, 2, 3, 7, 6, 2, 3, 1, 0, 2, 1, 0, 2, 6, 7, 6, 9, 6, 7, 3, 7, 6, 2, 5, 2, 6, 7, 3, 7, 3, 1, 2]
[6, 7, 3, 7, 6, 10, 8, 9, 6, 9, 8, 5, 8, 5, 2, 3, 1, 3, 7, 3, 2, 0, 1, 3, 2, 0, 2, 0, 2, 3, 2, 4, 2, 6, 10, 6, 9, 6, 2, 6, 9, 6, 9, 6, 9, 8, 5, 8, 5, 2, 3, 7, 3, 1, 0, 2, 1, 2, 0, 1, 2, 0, 2, 4, 2, 6, 2, 1, 3, 1, 2, 5, 2, 3, 1, 2, 5, 2, 5, 2, 1]
[4, 2

In [7]:
cityn = len(adj) - 1
pop_size = 20
gen_size = 80
epochs = 600
elitism_coef = 2

while True:
    pop = first_generation(pop_size, gen_size)
    for i in tqdm(range(epochs)):
        pop = next_generation(pop, pop_size, elitism_coef)
    # Sort population based on fitness
    pop.sort(key=fitness, reverse=True)
    # Calculate fitness values
    fit = pop_fitness(pop)
    # Check termination condition
    if cities(pop)[0] == 12:
        break
# Print the best solution
print(solution(pop[0]))
print(cities(pop)[0])
print(fitness(pop[0]))

100%|██████████| 600/600 [00:00<00:00, 659.38it/s]

Roma -> Milan -> Lyon -> Paris -> Madrid -> Barcelona -> Paris -> London -> Paris -> Brussels -> Amsterdam -> Cologne -> Frankfurt -> Cologne -> Frankfurt -> Milan -> Frankfurt -> Berlin
12
2457
